# Task
Viết code thực hiện A/B testing cho dữ liệu "ab_test_data.csv".

## Tải dữ liệu

### Subtask:
Tải dữ liệu từ tệp `ab_test_data.csv` vào một DataFrame của pandas.


**Reasoning**:
The first step is to import the pandas library to work with DataFrames.



**Reasoning**:
Load the data from the 'ab_test_data.csv' file into a pandas DataFrame named 'df'.



In [2]:
import numpy as np
import pandas as pd
import os

# Construct the path to the data file in an OS-independent way
data_dir='data'
filename='ab_test_data.csv'
# Get the current working directory of the script
current_dir = os.getcwd()

# Construct the path to the data file by going up one level ('..')
# and then into the 'data' folder.
file_path = os.path.join(current_dir, '..', '..', data_dir, filename)

# Normalize the path to resolve '..' and get the absolute path
file_path = os.path.abspath(file_path)

df = pd.read_csv(file_path) # đọc file csv với dấu chấm phẩy để ngăn cách các field và dấu phẩy thập phân
display(df)
df.info()  # hiển thị thông tin về dataframe


,user_id,group,converted
0,1,A,0
1,2,A,0
2,3,A,0
3,4,A,1
4,5,A,0
...,...,...,...
1945,1946,B,0
1946,1947,B,0
1947,1948,B,0
1948,1949,B,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1950 entries, 0 to 1949
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    1950 non-null   int64 
 1   group      1950 non-null   object
 2   converted  1950 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 45.8+ KB


## Khám phá dữ liệu

### Subtask:
Xem qua dữ liệu để hiểu cấu trúc và nội dung của nó.


**Reasoning**:
Display the first 5 rows, the info, and the unique values of the 'group' column of the DataFrame to understand its structure and content.



In [3]:
display(df.head())
display(df.info())
display(df['group'].nunique())

,user_id,group,converted
0,1,A,0
1,2,A,0
2,3,A,0
3,4,A,1
4,5,A,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1950 entries, 0 to 1949
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    1950 non-null   int64 
 1   group      1950 non-null   object
 2   converted  1950 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 45.8+ KB


None

2

## Phân tích dữ liệu theo nhóm

### Subtask:
Chia dữ liệu thành hai nhóm (A và B) và tính toán các số liệu thống kê mô tả cho từng nhóm (ví dụ: số lượng người dùng, số lượng chuyển đổi).


**Reasoning**:
Group the data by 'group' and calculate the count of users and the sum of conversions for each group.



In [4]:
group_stats = df.groupby('group')['converted'].agg(['count', 'sum'])
group_stats = group_stats.rename(columns={'count': 'user_count', 'sum': 'converted_count'})
display(group_stats)

,user_count,converted_count
group,,
A,1000,122
B,950,154


## Kiểm định giả thuyết

### Subtask:
Thực hiện kiểm định thống kê (ví dụ: kiểm định z hoặc kiểm định t cho tỷ lệ) để xác định xem có sự khác biệt có ý nghĩa thống kê giữa tỷ lệ chuyển đổi của nhóm A và nhóm B hay không.


**Reasoning**:
Import the necessary statistical library and extract the data for the z-test.



In [6]:
from statsmodels.stats.proportion import proportions_ztest

count = group_stats['converted_count']
display(count)
nobs = group_stats['user_count']
display(nobs)

group
A    122
B    154
Name: converted_count, dtype: int64

group
A    1000
B     950
Name: user_count, dtype: int64

**Reasoning**:
Perform the z-test for proportions using the extracted data and store the results.



In [8]:
z_stat, p_value = proportions_ztest(count, nobs)

print(f"Z-statistic: {z_stat}")
print(f"P-value: {p_value}")

Z-statistic: -2.5394999630829806
P-value: 0.011101105703581382


## Diễn giải kết quả

### Subtask:
Diễn giải kết quả kiểm định thống kê và đưa ra kết luận về kết quả A/B testing.


**Reasoning**:
Based on the calculated p-value and a standard alpha level of 0.05, determine if the null hypothesis should be rejected and interpret the A/B test results in context.



In [9]:
alpha = 0.05

print(f"Alpha (significance level): {alpha}")

if p_value < alpha:
    print("Kết luận: Bác bỏ giả thuyết null.")
    print("Có sự khác biệt có ý nghĩa thống kê về tỷ lệ chuyển đổi giữa nhóm A và nhóm B.")
    print("Trong ngữ cảnh A/B testing: Nhóm B có tỷ lệ chuyển đổi cao hơn đáng kể so với nhóm A.")
    print("Hành động đề xuất: Triển khai thay đổi của nhóm B cho tất cả người dùng.")
else:
    print("Kết luận: Không bác bỏ giả thuyết null.")
    print("Không có sự khác biệt có ý nghĩa thống kê về tỷ lệ chuyển đổi giữa nhóm A và nhóm B.")
    print("Trong ngữ cảnh A/B testing: Sự khác biệt về tỷ lệ chuyển đổi giữa hai nhóm có thể do ngẫu nhiên.")
    print("Hành động đề xuất: Có thể cần thu thập thêm dữ liệu hoặc xem xét các yếu tố khác.")

Alpha (significance level): 0.05
Kết luận: Bác bỏ giả thuyết null.
Có sự khác biệt có ý nghĩa thống kê về tỷ lệ chuyển đổi giữa nhóm A và nhóm B.
Trong ngữ cảnh A/B testing: Nhóm B có tỷ lệ chuyển đổi cao hơn đáng kể so với nhóm A.
Hành động đề xuất: Triển khai thay đổi của nhóm B cho tất cả người dùng.


## Summary:

### Data Analysis Key Findings

*   Group A had 1000 users with 122 conversions.
*   Group B had 950 users with 154 conversions.
*   The calculated Z-statistic from the proportions z-test is -2.539.
*   The calculated P-value from the proportions z-test is 0.0111.
*   The P-value (0.0111) is less than the significance level (alpha) of 0.05.

### Insights or Next Steps

*   Based on the statistically significant difference (P-value < 0.05), the null hypothesis is rejected. This indicates that Group B has a statistically significantly higher conversion rate compared to Group A.
*   The recommended action is to implement the changes introduced to Group B for all users, as they led to a higher conversion rate.
